# Bayes分类器

## 库导入

主要使用keras，TensorFlow(1版)

In [1]:
import sys
import time
import os
import re
import numpy as np
import pandas as pd
import pickle
import jieba

from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder

# 实时更新进度条
def print_flush(print_string):
    print(print_string, end='\r')
    sys.stdout.flush()

predictor_set = ["THUCNews", "TouTiao"]
name_of_predictor = predictor_set[1]

## 文件导入（CNN内有保存）

In [16]:
pickleFilePath = "./" + name_of_predictor + '/content_list/content_list.pickle'
with open(pickleFilePath, 'rb') as file:
    content_list = pickle.load(file)
    file.close()
pickleFilePath = "./" + name_of_predictor + '/label_list.pickle'
with open(pickleFilePath, 'rb') as file:
    label_list = pickle.load(file)
    file.close()
print("Load complete, {0} contents loaded, {1} labels loaded".format(len(content_list), len(label_list)))

Load complete, 382688 contents loaded, 382688 labels loaded


In [35]:
dataFrame=pd.DataFrame({
        'label':label_list,
        'content':content_list
        }) 
pd.value_counts(label_list)

科技    41543
娱乐    39396
体育    37568
汽车    35785
游戏    29300
文化    28031
金融    27085
教育    27058
世界    26909
军事    24984
旅游    21422
农业    19322
房产    17672
民生     6273
股票      340
dtype: int64

## 分词

### 逐内容处理

In [4]:
zhPattern = re.compile('[\u4e00-\u9fa5]+') 

def segment_reshape(dataFrame):
    startTime = time.time()
    sample_quantity = dataFrame.shape[0]
    for i, row in dataFrame.iterrows():
        segments = []
        fileContent = row['content']
        segs = jieba.cut(fileContent)
        for seg in segs:
            if zhPattern.search(seg):
                segments.append(seg)
        row['content'] = ' '.join(segments)
        if (i + 1)%1000==0 or i==sample_quantity - 1:
            string_0 = 'Segment process: %d/ %d' %(i + 1, sample_quantity)
            string_1 = ' Processed: %.2f%%' %(i/sample_quantity*100)
            usedTime = time.time() - startTime
            string_2 = ' Time cost: %.2f秒' %usedTime
            print_string = string_0 + string_1 + string_2
            print_flush(print_string)
        
segment_reshape(dataFrame)    

Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/2b/swj1bjf912l41nfy725k11ph0000gn/T/jieba.cache
Loading model cost 0.649 seconds.
Prefix dict has been built successfully.


### 停止词列表导入 Github: goto456/stopwords

In [5]:
stopWordsPath = "./stopwords-master/cn_stopwords.txt"
stopWords = pd.read_csv(stopWordsPath,
                        encoding='utf8', 
                        names = ["stopword"],
                        quoting=3,
                        sep="\t")

### 移除停止词

In [6]:
# r' 防转义
countVectorizer = CountVectorizer(stop_words=stopWords["stopword"].values.tolist(),
                                  min_df=0, token_pattern=r'\b\w+\b')

text_vector = countVectorizer.fit_transform(dataFrame['content'])

### 全现有文件序列化

In [7]:
pickleFilePath = './' + name_of_predictor + '/label_content_dataFrame.pickle'
with open(pickleFilePath, 'wb') as file:
    pickle.dump(dataFrame, file)
print("Label and content dataframe serialization finished.")

pickleFilePath = './' + name_of_predictor + '/content_vector.pickle'
with open(pickleFilePath, 'wb') as file:
    pickle.dump(text_vector, file)
print("Content vector serialization finished.")

Label and content dataframe serialization finished.
Content vector serialization finished.


## 模型建立

### 全现有文件导入

In [2]:
pickleFilePath = './' + name_of_predictor + '/label_content_dataFrame.pickle'
with open(pickleFilePath, 'rb') as file:
    dataFrame = pickle.load(file)
print("Label and content dataframe load finished.")

pickleFilePath = './' + name_of_predictor + '/content_vector.pickle'
with open(pickleFilePath, 'rb') as file:
    text_vector = pickle.load(file)
print("Content vector load finished.")

Label and content dataframe load finished.
Content vector load finished.


### 结果标签集

In [36]:
le = LabelEncoder()
label_encode = le.fit_transform(dataFrame['label'])

### 训练集测试集划分

In [48]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(text_vector, label_encode, random_state=24612781)

### 朴素多项式贝叶斯 (Scikit Learn)

In [49]:
bayesModel = MultinomialNB()
# 对标签label进行训练
bayesModel.fit(X_train, y_train)
score = bayesModel.score(X_train, y_train)
print("Bayes fit complete!")
print("Score on mean accuracy on the given test data and labels: {0:.6f} / {1:.6f}".format(score, 1))

Bayes fit complete!
Score on mean accuracy on the given test data and labels: 0.915085 / 1.000000


### 模型保存

In [52]:
import joblib
modelFilePath = "./" + name_of_predictor + "/trained_model/MultinomialNB.pkl"
leFilePath = "./" + name_of_predictor + "/trained_model/MultinomialNB_LE.pkl"
cvFilePath = "./" + name_of_predictor + "/trained_model/MultinomialNB_CV.pkl"

joblib.dump(bayesModel, modelFilePath) 
joblib.dump(le, leFilePath) 
joblib.dump(countVectorizer, cvFilePath) 

print("MNB model saved.")

MNB model saved.


### 模型加载

In [5]:
import joblib

modelFilePath = "./" + name_of_predictor + "/trained_model/MultinomialNB.pkl"
leFilePath = "./" + name_of_predictor + "/trained_model/MultinomialNB_LE.pkl"
cvFilePath = "./" + name_of_predictor + "/trained_model/MultinomialNB_CV.pkl"

bayesModel = joblib.load(modelFilePath)
le = joblib.load(leFilePath)
countVectorizer = joblib.load(cvFilePath)

## 预测和检验

In [57]:
y_pred = bayesModel.predict(X_test)
pred_label = le.inverse_transform(y_pred)
test_label = le.inverse_transform(y_test)

from sklearn.metrics import confusion_matrix

cm = pd.DataFrame(confusion_matrix(test_label, pred_label),
            columns=le.classes_, index=le.classes_)

if not os.path.isdir("./results/" + name_of_predictor):
    os.makedirs("./results/" + name_of_predictor)
cm.to_csv("./results/" + name_of_predictor + '/' + name_of_predictor + '_MNB_CM.csv')

cm

,世界,体育,军事,农业,娱乐,房产,教育,文化,旅游,民生,汽车,游戏,科技,股票,金融
世界,5571,48,398,41,144,15,45,32,139,32,56,23,98,0,82
体育,47,8851,23,15,180,9,80,26,49,1,38,80,46,0,21
军事,532,20,5225,13,47,3,65,36,29,4,34,73,62,0,22
农业,45,2,14,4216,31,38,51,45,112,34,37,9,41,0,118
娱乐,70,59,48,26,9163,6,52,139,56,58,30,113,71,0,15
房产,23,4,6,49,11,4102,28,15,54,4,19,3,37,0,88
教育,35,12,10,34,97,26,6073,113,29,9,15,31,141,0,34
文化,73,23,37,83,180,19,178,6082,232,21,17,54,66,0,40
旅游,157,23,22,93,74,65,43,135,4524,6,93,30,59,0,26
民生,21,7,13,24,173,10,16,49,13,1214,19,17,17,0,5


### 报告表

In [56]:
from sklearn.metrics import precision_recall_fscore_support

def eval_model(y_true, y_pred, labels):
    # 计算每个分类的Precision, Recall, f1, support
    p, r, f1, s = precision_recall_fscore_support(y_true, y_pred)
    # 计算总体的平均Precision, Recall, f1, support
    tot_p = np.average(p, weights=s)
    tot_r = np.average(r, weights=s)
    tot_f1 = np.average(f1, weights=s)
    tot_s = np.sum(s)
    res1 = pd.DataFrame({
        u'Label': labels,
        u'Precision': p,
        u'Recall': r,
        u'F1': f1,
        u'Support': s
    })
    res2 = pd.DataFrame({
        u'Label': ['总体'],
        u'Precision': [tot_p],
        u'Recall': [tot_r],
        u'F1': [tot_f1],
        u'Support': [tot_s]
    })
    res2.index = [999]
    res = pd.concat([res1, res2])
    res.to_csv("./results/" + name_of_predictor + '/' + name_of_predictor + '_MNB_PRFS.csv')
    return res[['Label', 'Precision', 'Recall', 'F1', 'Support']]

eval_model(test_label, pred_label, le.classes_)

,Label,Precision,Recall,F1,Support
0,世界,0.820955,0.828525,0.824722,6724
1,体育,0.962170,0.935031,0.948406,9466
2,军事,0.879482,0.847526,0.863208,6165
3,农业,0.876507,0.879616,0.878059,4793
4,娱乐,0.882500,0.924995,0.903248,9906
5,房产,0.916648,0.923250,0.919937,4443
6,教育,0.881039,0.911999,0.896251,6659
7,文化,0.904252,0.856017,0.879474,7105
8,旅游,0.827511,0.845607,0.836461,5350
9,民生,0.868383,0.759700,0.810414,1598


### 人工输入预测

In [25]:
def predict(input_content):
    for i in range(len(input_content)):
        input_content[i] = ' '.join(jieba.cut(input_content[i]))
    input_seg_vec = countVectorizer.transform(input_content)
    
    return le.inverse_transform(bayesModel.predict(input_seg_vec))

def predict_latent(input_content):
    for i in range(len(input_content)):
        input_content[i] = ' '.join(jieba.cut(input_content[i]))
    input_seg_vec = countVectorizer.transform(input_content)
    predict_proba = bayesModel.predict_proba(input_seg_vec)
    latent = np.argsort(np.max(predict_proba, axis=0))[-2]
    return le.inverse_transform([latent])
    
input_content = ["""

4月29日发射入轨的空间站天和核心舱，近日先后完成交会对接、航天员驻留、机械臂等平台功能测试，以及空间应用项目设备在轨性能检查，各项功能正常、运行状态良好，已进入交会对接轨道，后续将继续开展与天舟二号货运飞船交会对接的准备工作。
"""]

print("Predict: " + predict(input_content)[0])
print("Latent predict: " + predict_latent(input_content)[0])

Predict: 科技
Latent predict: 金融


### 新浪微博预测

In [27]:
import requests

headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 11_0_0) '
                         'AppleWebKit/537.36 (KHTML, like Gecko) '
                         'Chrome/86.0.4240.198 Safari/537.36'}
html = requests.get('https://s.weibo.com/top/summary?cate=realtimehot', headers=headers)
page = html.content.decode()

res = re.findall(
    r'<tr class="">\s*<td class="td-01 ranktop">(\d+)<\/td>\s*<td class="td-02">\s*<a href="\/('
    r'.*?)&.*?target="_blank">(.*?)<\/a>', page, re.S)

# ind = [i[0] for i in res]
# link = ['https://s.weibo.com/' + i[1] for i in res]
title = [i[2] for i in res]
pred = [predict([t])[0] for t in title]
pred_latent = [predict_latent([t])[0] for t in title]


dataFrame = pd.DataFrame({
        u'热搜标题': title,
        u'预测结果': pred,
        u'备用结果': pred_latent
    })
dataFrame.index = [i + 1 for i in range(50)]

dataFrame[['热搜标题', '预测结果', '备用结果']]

,热搜标题,预测结果,备用结果
1,只需打1针的新冠疫苗,农业,金融
2,白敬亭给倪妮写的祝福,娱乐,教育
3,大多数主播月收入3000至5000元,游戏,金融
4,吴世勋吴倩吻戏,娱乐,文化
5,唐九洲换头像,科技,汽车
6,反穿内衣打疫苗,娱乐,世界
7,外卖骑手设置呼叫转移给110,科技,汽车
8,巴勒斯坦男子乞求废墟中母亲别死,世界,军事
9,韩版乘风破浪的姐姐 退役女团成员,娱乐,体育
10,深圳应急管理回应赛格大厦摇晃,金融,教育
